In [1]:
import torch
from tqdm.notebook import tqdm


from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification


In [2]:
import pandas as pd
df = pd.read_csv('processed_lyrics.csv')


In [3]:
df = df.groupby('Genre', group_keys=False).apply(lambda s: s.sample(df['Genre'].value_counts().min())) #
df['Genre'].value_counts()

Hip Hop    14796
Pop        14796
Rock       14796
Name: Genre, dtype: int64

Since the Genres are imbalanced we use this as 

In [4]:

possible_labels = df.Genre.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Hip Hop': 0, 'Pop': 1, 'Rock': 2}

In [5]:
df['label'] = df.Genre.replace(label_dict)

In [6]:
from sklearn.model_selection import train_test_split
 
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.2,
    random_state=42,
    stratify=df.label.values
)

In [7]:
X_train

array([88946, 16505, 87736, ...,  4572, 29197, 93936])

In [8]:
['not_set']*df.shape[0]

['not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'no

In [9]:
#Adding a column with 'not_set'
df['data_type'] = ['not_set']*df.shape[0]

#Setting the train/validation values
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Genre', 'label', 'data_type']).count()

SName  Lyric  Artist  lyric_length
Genre   label data_type                                    
Hip Hop 0     train      11837  11837   11837         11837
              val         2959   2959    2959          2959
Pop     1     train      11837  11837   11837         11837
              val         2959   2959    2959          2959
Rock    2     train      11836  11836   11836         11836
              val         2960   2960    2960          2960

In [10]:

## TOKENIZE THE DATA
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Lyric.values, 
    add_special_tokens=True,  # sequences will be encoded with the special tokens relative to their model.
    return_attention_mask=True, # batching sequences together. will return the attention mask according to the specific tokenizer defined by the max_length attribute.
    pad_to_max_length=True, # pad all the titles to certain maximum length
    max_length=256, 
    return_tensors='pt' # return PyTorch
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Lyric.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/martinfriberg/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2221: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [11]:
# split the data into input_ids, attention_masks and labels.
# after we get encoded data set, we can create training data and validation data.
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

### BERT Pre-trained Model
We are treating each Genre as its unique sequence, so one sequence will be classified to one of the three labels

In [14]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", # smaller pre-trained model.
                                                      num_labels=len(label_dict), # number of output labels.
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### Data Loaders 
DataLoader combines a dataset and a sampler and provides an iterable over the given dataset
RandomSampler is used for training and SequentialSampler for validation.
Batch_size is set after the available memory. 
#### Start with batch_size=3? Increase when using cloud computing

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

# Optimizer & Scheduler
- To construct an optimizer, we have to give it an iterable containing the parameters to optimize. Then, we can specify optimizer-specific options such as the learning rate, epsilon, etc.
- Try with 5 epochs and iterate
- Create a schedule with a learning rate that decreases linearly from the initial learning rate set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial learning rate set in the optimizer.

In [15]:

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# Performance Metrics
We will use f1 score and accuracy per class as performance metrics.


In [16]:

from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [20]:
torch.cuda.is_available()

False

# Training Loop

In [18]:
import random
import numpy as np
from sklearn.metrics import classification_report

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# In case you are running on gpu, you also need to set the model: model.to(device) - Is this set in the next step? 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
    
    
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/home/ubuntu/NLP{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    print(classification_report(true_vals, predictions))
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/11837 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Loading and Evaluating the Model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('/home/ubuntu/NLP5.model', map_location=torch.device('cuda')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)
print(classification_report(true_vals, predictions))